##Preprocessing

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/KrisnaPinasthika/textmining_A_2021/main/dataset.csv", encoding="UTF-8")

# 1. Parsing (1 Komentar menjadi 1 dokumen)
df

,Nama,Komentar
0,Anon,Indonesia bangga punya Greysia Polii dan Apriy...
1,Anon,"Indonesi bangga, dulu bendera Jepang berkibar ..."
2,Anon,ikut menangis karena bangga...
3,Anon,"Selamat untuk Greysia P dan Apriyani R, kalian..."
4,Anon,Satu kata luar biasa
...,...,...
3009,Wong Ndamen,Ayo bisa trending 1 masa kala sama plasti*
3010,Dyah Setiarini,Mkshya ak senang deh ak senang bngt deh ak aja...
3011,Abdul Wahab,Hapus emot gass kita trendint
3012,Rez' syafiQ,Adakah yg sengaja gk nnton biar menang??


In [2]:
# 2. Lexical Analysis (Lexing / Tokenization)

import re
import string

def hapusLinkTagDuit(text):
    patt1 = r"https\S+|http\S+|@\S+|#\S+|\$\S+|\.\S+|\d"
    return " ".join([re.sub(patt1, "", i) for i in text.split()]).replace("  ", " ").strip()

def hapusTandaBaca(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [3]:
# 2.1 Cleaning
df["Komentar"] = df["Komentar"].apply(lambda x: hapusLinkTagDuit(x)) \
                                .apply(lambda x: hapusTandaBaca(x)) \
                                .apply(lambda x: x.lower()) \
                                .apply(lambda x: remove_emoji(x)) 

In [4]:
# Replace kata-kata yang berpotensi salah

df["Komentar"].str.replace("indonesia", "wokwow")

0       wokwow bangga punya greysia polii dan apriyani...
1       indonesi bangga dulu bendera jepang berkibar d...
2                             ikut menangis karena bangga
3       selamat untuk greysia p dan apriyani r kalian ...
4                                    satu kata luar biasa
                              ...                        
3009              ayo bisa trending masa kala sama plasti
3010    mkshya ak senang deh ak senang bngt deh ak aja...
3011                        hapus emot gass kita trendint
3012               adakah yg sengaja gk nnton biar menang
3013                             komentator terburuk asli
Name: Komentar, Length: 3014, dtype: object

In [5]:
df

,Nama,Komentar
0,Anon,indonesia bangga punya greysia polii dan apriy...
1,Anon,indonesi bangga dulu bendera jepang berkibar d...
2,Anon,ikut menangis karena bangga
3,Anon,selamat untuk greysia p dan apriyani r kalian ...
4,Anon,satu kata luar biasa
...,...,...
3009,Wong Ndamen,ayo bisa trending masa kala sama plasti
3010,Dyah Setiarini,mkshya ak senang deh ak senang bngt deh ak aja...
3011,Abdul Wahab,hapus emot gass kita trendint
3012,Rez' syafiQ,adakah yg sengaja gk nnton biar menang


In [6]:
# 2.2 Tokenizing
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="x", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(df["Komentar"])

In [7]:
tokenizer.word_counts

OrderedDict([('indonesia', 1125),
             ('bangga', 532),
             ('punya', 70),
             ('greysia', 225),
             ('polii', 79),
             ('dan', 586),
             ('apriyani', 213),
             ('rahayu', 134),
             ('indonesi', 6),
             ('dulu', 26),
             ('bendera', 15),
             ('jepang', 18),
             ('berkibar', 23),
             ('di', 332),
             ('tanah', 14),
             ('air', 100),
             ('kita', 204),
             ('kini', 6),
             ('sangsaka', 2),
             ('merah', 22),
             ('putih', 20),
             ('berkibarlah', 8),
             ('ku', 51),
             ('terima', 200),
             ('kasih', 218),
             ('greysapoli', 2),
             ('ikut', 105),
             ('menangis', 81),
             ('karena', 41),
             ('selamat', 298),
             ('untuk', 283),
             ('p', 5),
             ('r', 2),
             ('kalian', 375),
             ('sang

In [8]:
# 3. Stopwords Removal (Filtering)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopIndo = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
temp_data = []
for i in tokenizer.word_index:
    if i in stopIndo:
        temp_data.append(i)

for i in temp_data:
    if i in tokenizer.word_index:
        tokenizer.word_index.pop(i)

In [10]:
# 4. Stemming
!pip install Sastrawi

     |████████████████████████████████| 209 kB 5.1 MB/s 


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stem
token_words = tokenizer.word_counts

stem_collection = []

for i in token_words:
    t = stemmer.stem(i)
    stem_collection.append(t)

In [ ]:
stem_words = {}

for i, j in zip(token_words, stem_collection):
    if j in stem_words:
        stem_words[j] += token_words[i]
    else:
        stem_words[j] = token_words[i]
    # print(f"Token : {i} {token_words[i]}, stem = {j}")
    # print(f"Hasil stem : {stem_words[j]}")

In [ ]:
dict_items = stem_words.items()
sorted_stem = sorted(dict_items)

In [ ]:
sorted_stem

## Pembobotan Kata